In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all" # to run all arguments and not just the last one
from IPython.display import Image
%matplotlib inline

In [4]:
# Ensemble modelling is about creating a class of classifiers which have a better predictive performance than
# the individual models. The book focusses on popular ensemble models that use the majority voting principle
# generalized to multi-class settings, majority vote method is called plural voting
from scipy.misc import comb
import math

In [ ]:
def ensemble_error